In [27]:
"""
Script para entrenamiento, evaluación comparativa y selección de modelos de
clasificación para predecir la calidad del aire.
"""

# 1. IMPORTACIÓN DE LIBRERÍAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import joblib
from pathlib import Path
import time

In [28]:
# 2. CONFIGURACIÓN Y CARGA DE DATOS
# Definir rutas de entrada y salida
BASE_PATH = Path(r"B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Codigos")
OUTPUT_PATH = BASE_PATH / "Resultados_Modelos"
OUTPUT_PATH.mkdir(exist_ok=True) # Crear carpeta de resultados si no existe

# Cargar los datasets pre-procesados
print(">>> Cargando datasets...")
X_train = pd.read_csv(BASE_PATH / "X_train_balanced.csv")
y_train = pd.read_csv(BASE_PATH / "y_train_balanced.csv").values.ravel() # .ravel() convierte el DataFrame a un array 1D
X_test = pd.read_csv(BASE_PATH / "X_test.csv")
y_test = pd.read_csv(BASE_PATH / "y_test.csv").values.ravel()

print(f"Entrenando con: {X_train.shape[0]} ejemplos (Balanceados)")
print(f"Testeando con:  {X_test.shape[0]} ejemplos (Reales)")


>>> Cargando datasets...
Entrenando con: 96920 ejemplos (Balanceados)
Testeando con:  14911 ejemplos (Reales)


In [29]:
# 3. DEFINICIÓN DE MODELOS A COMPARAR
# Se definen los modelos con parámetros iniciales. n_jobs=-1 usa todos los cores de CPU disponibles.
modelos = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    "KNN": KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42, n_jobs=-1)
}

# Diccionario para guardar los resultados de cada modelo
resultados = {}

In [30]:
# 4. BUCLE DE ENTRENAMIENTO Y EVALUACIÓN
print("\n--- INICIANDO ENTRENAMIENTO Y EVALUACIÓN ---")
for nombre, modelo in modelos.items():
    print(f"\n{'='*40}\nMODELO: {nombre}\n{'='*40}")

    # Entrenar el modelo con los datos de entrenamiento balanceados
    modelo.fit(X_train, y_train)

    # Evaluar precisión en datos de entrenamiento (mide memorización)
    y_train_pred = modelo.predict(X_train)
    train_acc = accuracy_score(y_train, y_train_pred)

    # Evaluar precisión en datos de prueba (mide capacidad de generalización)
    y_test_pred = modelo.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)

    print(f"-> Precisión en Entrenamiento (Memorización): {train_acc:.2%}")
    print(f"-> Precisión en Prueba (Generalización):    {test_acc:.2%}")

    # Comprobar posible sobreajuste
    diff = train_acc - test_acc
    if diff > 0.15:
        print(f"   [ALERTA] Posible sobreajuste (Diferencia: {diff:.2%})")
    else:
        print("   [OK] El modelo parece generalizar bien.")

    # Guardar resultados y predicciones para análisis posteriores
    resultados[nombre] = {
        'modelo_obj': modelo,
        'predicciones': y_test_pred,
        'probabilidades': modelo.predict_proba(X_test) if hasattr(modelo, "predict_proba") else None
    }

    # Generar y guardar la matriz de confusión para este modelo
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_test_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
    plt.title(f"Matriz de Confusión: {nombre}\nAccuracy: {test_acc:.2%}")
    plt.xlabel('Predicción del Modelo')
    plt.ylabel('Clase Real')
    plt.tight_layout()
    plt.savefig(OUTPUT_PATH / f"matriz_confusion_{nombre.replace(' ', '_')}.png", dpi=200)
    plt.close()

print("\n--- EVALUACIÓN COMPLETADA ---")



--- INICIANDO ENTRENAMIENTO Y EVALUACIÓN ---

MODELO: Random Forest
-> Precisión en Entrenamiento (Memorización): 100.00%
-> Precisión en Prueba (Generalización):    86.65%
   [OK] El modelo parece generalizar bien.

MODELO: KNN
-> Precisión en Entrenamiento (Memorización): 96.27%
-> Precisión en Prueba (Generalización):    81.18%
   [ALERTA] Posible sobreajuste (Diferencia: 15.09%)

MODELO: XGBoost
-> Precisión en Entrenamiento (Memorización): 97.22%
-> Precisión en Prueba (Generalización):    86.23%
   [OK] El modelo parece generalizar bien.

--- EVALUACIÓN COMPLETADA ---


In [31]:
# 5. GRÁFICO COMPARATIVO DE CURVAS ROC (MULTICLASE)
print("\nGenerando gráfico comparativo de Curvas ROC...")
plt.figure(figsize=(12, 8))
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=range(n_classes))

for nombre, res in resultados.items():
    y_prob = res['probabilidades']
    if y_prob is None: continue

    # Calcular ROC y AUC para cada clase y luego promediar (macro-average)
    fpr, tpr, roc_auc = {}, {}, {}
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Interpolar todas las curvas ROC en puntos comunes
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    macro_auc = auc(all_fpr, mean_tpr)

    plt.plot(all_fpr, mean_tpr, label=f'{nombre} (Macro AUC = {macro_auc:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Azar (AUC = 0.50)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (1 - Especificidad)')
plt.ylabel('Tasa de Verdaderos Positivos (Sensibilidad/Recall)')
plt.title('Comparación de Curvas ROC (Promedio Macro Multiclase)')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.4)
plt.tight_layout()
plt.savefig(OUTPUT_PATH / "comparacion_curvas_roc.png", dpi=300)
plt.close()
print(f"Gráfico ROC guardado en '{OUTPUT_PATH}'")


Generando gráfico comparativo de Curvas ROC...
Gráfico ROC guardado en 'B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Codigos\Resultados_Modelos'


In [32]:
# 6. ANÁLISIS DE IMPORTANCIA DE VARIABLES (PARA EL MEJOR MODELO)
print("\nGenerando gráfico de importancia de variables para XGBoost...")
modelo_xgboost = resultados['XGBoost']['modelo_obj']
importances = modelo_xgboost.feature_importances_
feature_names = X_train.columns
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 7))
plt.title("Importancia de Variables (Feature Importance) - XGBoost")
plt.bar(range(X_train.shape[1]), importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), feature_names[indices], rotation=45, ha="right")
plt.ylabel("Importancia Relativa")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(OUTPUT_PATH / "importancia_variables_xgboost.png", dpi=300)
plt.close()
print(f"Gráfico de importancia guardado en '{OUTPUT_PATH}'")


Generando gráfico de importancia de variables para XGBoost...
Gráfico de importancia guardado en 'B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Codigos\Resultados_Modelos'


In [33]:
# 7. GUARDADO DEL MODELO FINAL (SERIALIZACIÓN)
print("\nGuardando el modelo final (XGBoost)...")
ruta_modelo_final = OUTPUT_PATH / "modelo_xgboost_final.joblib"
joblib.dump(modelo_xgboost, ruta_modelo_final)

print("\n[PROCESO COMPLETADO]")
print(f"Modelo final guardado en: '{ruta_modelo_final}'")


Guardando el modelo final (XGBoost)...

[PROCESO COMPLETADO]
Modelo final guardado en: 'B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Codigos\Resultados_Modelos\modelo_xgboost_final.joblib'


In [34]:
# 5. REPORTES DE CLASIFICACIÓN DETALLADOS
print("\n--- REPORTES DE CLASIFICACIÓN DETALLADOS (SOBRE CONJUNTO DE PRUEBA) ---")

# Definir los nombres de las clases para que el reporte sea legible
# El orden corresponde a las etiquetas numéricas 0, 1, 2, 3, 4
nombres_clases = ['Muy Bueno', 'Bueno', 'Regular', 'Malo', 'Muy Malo']

for nombre, res in resultados.items():
    print(f"\n{'='*60}")
    print(f"  Reporte Detallado para: {nombre}")
    print(f"{'='*60}")

    # Obtener las predicciones del diccionario de resultados
    y_pred = res['predicciones']

    # Generar y mostrar el reporte de clasificación
    # Este reporte incluye precisión, recall y f1-score para cada clase.
    reporte = classification_report(y_test, y_pred, target_names=nombres_clases)
    print(reporte)


--- REPORTES DE CLASIFICACIÓN DETALLADOS (SOBRE CONJUNTO DE PRUEBA) ---

  Reporte Detallado para: Random Forest
              precision    recall  f1-score   support

   Muy Bueno       0.93      0.90      0.92      8308
       Bueno       0.75      0.76      0.75      3501
     Regular       0.72      0.79      0.75      1034
        Malo       0.88      0.90      0.89      1217
    Muy Malo       0.97      0.98      0.97       851

    accuracy                           0.87     14911
   macro avg       0.85      0.87      0.86     14911
weighted avg       0.87      0.87      0.87     14911


  Reporte Detallado para: KNN
              precision    recall  f1-score   support

   Muy Bueno       0.92      0.83      0.88      8308
       Bueno       0.63      0.73      0.67      3501
     Regular       0.61      0.74      0.67      1034
        Malo       0.85      0.85      0.85      1217
    Muy Malo       0.95      0.97      0.96       851

    accuracy                           0